# Intelligent Water Drops

Cada gota representa una solución al problema.
Entorno: grafo del problema del viajante simétrico a resolver.
Se representa como una matriz de adyacencia en la que el valor de la posición [i][j] es la distancia entre las ciudades i y j.

In [ ]:
import random
import copy
import json
import time
import tsplib95

### Función cargaCasoDePrueba

Lee un caso de prueba del fichero con nombre _nombreFichero_ y devuelve la matriz de adyacencia correspondiente.

In [ ]:
def cargaCasoDePrueba(nombreFichero):
    with open(nombreFichero, 'r') as archivo:
        matrizAdy = json.loads(archivo.read())
    return matrizAdy

### Función tsplibAMatrizAdyacencia

Convierte el problema dado del formato tsplib a la matriz de adyacencia correspondiente.

In [ ]:
def tsplibAMatrizAdyacencia(problema):
    matrizAdy = [[]] * problema.dimension
    for i in range(problema.dimension):
        matrizAdy[i] = [problema.get_weight(i+1,j+1) for j in range(problema.dimension)]
    return matrizAdy

### Función cargaCasoTsplib

Carga un problema de la tsplib y lo devuelve ya en forma de matriz de adyacencia.

In [ ]:
def cargaCasoTsplib(nombreFichero):
    problema = tsplib95.load(nombreFichero)
    return tsplibAMatrizAdyacencia(problema)

### Función guardarResultados

Dadas las ejecuciones de un algoritmo, las guarda en un fichero formateado adecuadamente para posteriormente cargarlas en el notebook Graficas.ipynb

In [ ]:
def guardarResultados(ejecuciones, problema, algoritmo, num):
    nombreFichero = problema + "_" + algoritmo + "_" + str(num) + ".txt"
    with open(nombreFichero, 'a') as archivo:
        for ejecucion in ejecuciones:
            archivo.write(str(ejecucion[0]) + '\n')
            for hormiga in ejecucion[1]:
                archivo.write(str(hormiga))
            archivo.write('FIN\n')

## Arista
La clase Arista nos permite representar un camino entre dos ciudades concretas. Ayuda a guardar el valor de _tierra_ que hay en cada momento en el camino.

In [ ]:
class Arista:
    def __init__(self, desde, hasta, distancia=None, tierraInicial=None):
        self.desde = desde
        self.hasta = hasta
        self.distancia = 1 if distancia is None else distancia
        self.tierraInicial = 10000 if tierraInicial is None else tierraInicial
        self.tierra = tierraInicial
    
    # Actualiza el valor de tierra de la arista al recorrerla una gota.
    def actualizarVariacionTierra(self, variacionTierra):
        self.tierra = (1-rho_n) * self.tierra - rho_n * variacionTierra

## Entorno
La clase Entorno representa una instancia del problema del viajante a resolver. Contiene una serie determinada de Aristas que crean el grafo a resolver.

In [ ]:
class Entorno:
    def __init__(self, matrizAdy, tierraInicial=None):
        self.conjuntoAristas = set()
        self.aristas = self.creaAristas(matrizAdy, tierraInicial)
        self.numVertices = len(matrizAdy)
        
    def creaAristas(self, matrizAdy, tierraInicial=None):
        aristas = {}
        for i in range(len(matrizAdy)):
            for j in range(i):
                arista = Arista(i, j, matrizAdy[i][j],
                                tierraInicial = tierraInicial)
                aristas[i,j] = arista
                aristas[j,i] = arista
                self.conjuntoAristas.add(arista)
        return aristas
    
    def resetearTierra(self):
        for arista in self.conjuntoAristas:
            arista.tierra = arista.tierraInicial

## GotaIWD
La clase GotaIWD representa una solución concreta al problema planteado en el entorno.
Realiza un trayecto concreto entre las ciudades eligiendo sus movimientos en base al nivel de _tierra_ de las aristas. Cuenta también con dos parámetros propios; el nivel de tierra que porta la gota y su velocidad.

In [ ]:
class GotaIWD:
    def __init__(self, entorno, velocidadInicial=None):
        self.nodoActual = None
        self.inicio = None
        self.entorno = entorno
        self.visitados = []
        self.porVisitar = []
        self.trayecto = []
        self.distancia = 0
        
        self.tierra = 0
        self.velocidad = 200 if velocidadInicial is None else velocidadInicial
        
    def __str__(self):
        resultado = str(self.distancia) + '\n' + str(self.visitados) + '\n'
        return resultado
    
    def __repr__(self):
        resultado = str(self.distancia) + '\n' + str(self.visitados) + '\n'
        return resultado
    
    # Nos va a interesar poder ordenar las gotas según la distancia de su solución
    # para poder ordenarlas y quedarnos con la mejor.
    def __eq__(self,other):
        return self.distancia == other.distancia
    
    def __lt__(self,other):
        return self.distancia < other.distancia
        
    def inicializar(self, inicio, velocidadInicial):
        self.entorno = self.entorno
        self.inicio = inicio
        self.nodoActual = inicio
        self.visitados = [inicio]
        self.porVisitar = [v for v in range(entorno.numVertices) if v != inicio]
        self.trayecto = []
        self.distancia = 0
        
        self.tierra = 0
        self.velocidad = velocidadInicial
        return self
    
    # Elige un movimiento para la gota y lo efectúa devolviendo la arista elegida.
    def movimiento(self):
        eleccion = self.elegirMovimiento()
        return self.realizaMovimiento(eleccion)
    
    # Elige el movimiento de la gota. Devuelve la arista a usar.
    def elegirMovimiento(self):
        # Sólo queda volver al nodo original
        if not self.porVisitar:
            return self.entorno.aristas[self.nodoActual, self.inicio]
        # Sólo queda un nodo por visitar
        if len(self.porVisitar) == 1:
            return self.entorno.aristas[self.nodoActual, self.porVisitar[0]]
        listaAristas = []
        for nodo in self.porVisitar:
            listaAristas.append(self.entorno.aristas[self.nodoActual, nodo])
        minTierra = min([arista.tierra for arista in listaAristas])
        listaf = [self.f(arista, minTierra) for arista in listaAristas]
        fAcumulado = sum(listaf)
        probabilidades = []
        for f in listaf:
            # Calculamos las probabilidades en %
            probabilidades.append(100 * f/fAcumulado)
        return random.choices(listaAristas, weights=probabilidades)[0]
    
    # Actualiza la posición de la IWD según la arista que va a seguir
    def realizaMovimiento(self, arista):
        hasta = arista.hasta if self.nodoActual == arista.desde else arista.desde
        if  not (hasta == self.inicio):
            self.visitados.append(hasta)
            self.porVisitar.remove(hasta)
        self.distancia += arista.distancia
        self.nodoActual = hasta
        return arista

    # Función auxiliar para la elección probabilista
    def f(self, arista, minTierra):
        return 1/(eps + self.g(arista, minTierra))
    
    # Función auxiliar para la elección probabilista
    def g(self, arista, minTierra):
        if minTierra >= 0:
            return arista.tierra
        return arista.tierra - minTierra
    
    # Actualiza la velocidad de la gota en función de la tierra de la arista
    def actualizarVelocidad(self, arista):
        self.velocidad = self.velocidad + (a_v/(b_v + c_v * pow(arista.tierra, 2)))
    
    # Calcula la tierra que la gota desprende de la arista y la actualiza.
    # Devuelve la variación de la tierra para actualizarla directamente en la arista sin recalcularla.
    def actualizarVariacionTierra(self, arista):
        variacionTierra = a_s / (b_s + c_s * pow(self.tiempo(arista),2))
        self.tierra = self.tierra + variacionTierra
        return variacionTierra
    
    # Función auxiliar para el cálculo de la variación de tierra
    def tiempo(self, arista):
        # En el caso del TSP, podemos usar la distancia como heurística.
        return arista.distancia/self.velocidad
    

### Función crearGotas

Esta función genera una nueva lista de gotas y les asigna el entorno asociado.

In [ ]:
def crearGotas(numGotas, entorno):
    gotas = []
    for i in range(numGotas):
        nuevaGota = GotaIWD(entorno)
        gotas.append(nuevaGota)
    return gotas

### Función reiniciarGotas

Esta función devuelve las gotas a su estado inicial de tierra y velocidad y las ubica en un punto inicial aleatorio del grafo.

In [ ]:
def reiniciarGotas(gotas, velocidadInicial):
    for gota in gotas:
        gota.inicializar(random.randrange(gota.entorno.numVertices), velocidadInicial)

### Función encontrarSoluciones

Realiza todos los movimientos de todas las gotas (actualizando sus valores de tierra y su velocidad) según lo indicado en Intelligent Water Drops para que cada gota tenga una solución completa. También gestiona las actualizaciones locales de las aristas.

In [ ]:
# Dos bucles: ya que todas las gotas tienen que moverse por los nodos hasta haberlos visitado todos.
def encontrarSoluciones(entorno, gotas):
    for i in range(entorno.numVertices):
        for gota in gotas:
            arista = gota.movimiento()
            gota.trayecto.append(arista)
            
            gota.actualizarVelocidad(arista)
            variacionTierra = gota.actualizarVariacionTierra(arista)
            arista.actualizarVariacionTierra(variacionTierra)
            
    gotas = sorted(gotas)
    return gotas[0]

In [ ]:
def actualizarAristasMejorActual(entorno, mejorSolucionActual):
    for arista in mejorSolucionActual.trayecto:
        arista.tierra = (1 + rho_iwd) * arista.tierra - rho_iwd * (1 / (entorno.numVertices - 1)) * mejorSolucionActual.tierra

### Funciones IWD

Dado un entorno, un número de iteraciones, y un número de gotas, aplica la metaheurística de Intelligent Water Drops para intentar obtener una buena solución al problema del TSP. Devuelve la mejor solución encontrada tras todas las iteraciones. Cada una usa una condición de parada diferente.

In [ ]:
def IWDTiempo(entorno, numGotas):
    tiempoInicio = time.time()
    listaSoluciones = []
    entorno.resetearTierra()
    mejorSolucionGlobal = None
    gotas = crearGotas(numGotas, entorno)
    # Paramos el algoritmo tras 300 segundos (5 minutos)
    while time.time() - tiempoInicio < 300:
        reiniciarGotas(gotas, velocidad_ini)
        mejorSolucionActual = encontrarSoluciones(entorno, gotas)
        if mejorSolucionGlobal is None or mejorSolucionActual < mejorSolucionGlobal:
            mejorSolucionGlobal = copy.deepcopy(mejorSolucionActual)
        listaSoluciones.append(mejorSolucionGlobal)
        actualizarAristasMejorActual(entorno, mejorSolucionActual)
    tiempoFin = time.time()
    tiempo = tiempoFin - tiempoInicio
    return listaSoluciones, tiempo

In [ ]:
def IWD(entorno, numGotas):
    tiempoInicio = time.time()
    # Si no se mejora la solución después de maxSeguidas iteraciones, se para el algoritmo.
    seguidas = 0
    maxSeguidas = 10000
    listaSoluciones = []
    entorno.resetearTierra()
    mejorSolucionGlobal = None
    gotas = crearGotas(numGotas, entorno)
    while seguidas < maxSeguidas:
        reiniciarGotas(gotas, velocidad_ini)
        mejorSolucionActual = encontrarSoluciones(entorno, gotas)
        if mejorSolucionGlobal is None or mejorSolucionActual < mejorSolucionGlobal:
            seguidas = 0
            mejorSolucionGlobal = copy.deepcopy(mejorSolucionActual)
        else:
            seguidas += 1
        listaSoluciones.append(mejorSolucionGlobal)
        actualizarAristasMejorActual(entorno, mejorSolucionActual)
    tiempoFin = time.time()
    tiempo = tiempoFin - tiempoInicio
    return listaSoluciones, tiempo

## Pruebas

#### Definiciones de cada parámetro:

a_v, b_v, c_v: Parámetros de actualización de la velocidad.

a_s, b_s, c_s: Parámetros de actualización de la tierra.

rho_n: Parámetro de actualización de tierra local.

rho_iwd: Parámetro de actualización de tierra global.

tierra_ini: Tierra inicial de cada arista.

velocidad_ini: Velocidad inicial de las gotas.

eps: Parámetro numérico positivo muy pequeño para evitar la división por cero.

In [ ]:
a_v = c_v = 1
b_v = 0.01
a_s = c_s = 1
b_s = 0.01

rho_n = rho_iwd = 0.9

tierra_ini = 10000
velocidad_ini = 200

eps = 0.000000000001

In [ ]:
# Cambiar por el nombre del problema deseado
nombreProblema = 'eil51'
matrizAdy = cargaCasoTsplib('ALL_tsp/' + nombreProblema + '.tsp')
# Alterar este range si se quiere ejecutar pruebas con varios números de individuos.
# Por defecto, se empieza por un número de individuos igual al número de ciudades,
# y se va aumentando de 5 en 5 hasta 5 veces esa cifra.
for numGotas in range(len(matrizAdy), 5 * len(matrizAdy), 5):
    # 20 ejecuciones con cada nº de gotas
    ejecuciones = []
    for i in range(20):
        entorno = Entorno(matrizAdy)
        soluciones, tiempo = IWD(entorno, numGotas)
        ejecuciones.append((tiempo, soluciones))
    guardarResultados(ejecuciones, nombreProblema, 'IWD', numGotas)